In [ ]:
# Either the Python file should be where the data is or the working directory should be set before running the code

In [ ]:
import os

In [ ]:
# Check working directory

os.getcwd()

In [ ]:
# Substitute the path to where the data is located with the "..."

os.chdir('...\\Data')

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

In [ ]:
data = ['2004.csv', '2005.csv']
main_df = pd.concat(pd.read_csv(filename, encoding = 'latin-1', 
                                usecols = ['Year', 'Month', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime', 'DepTime', 'ArrTime',
                                           'ArrDelay', 'DepDelay', 'TailNum', 'Origin', 'Dest']) 
                    for filename in data)

In [ ]:
main_df.info(memory_usage = 'deep')

In [ ]:
main_df

In [ ]:
main_df.isnull().sum()

In [ ]:
missing_values = main_df.isna()

In [ ]:
missing_values_sum = missing_values.sum()

In [ ]:
(missing_values_sum / len(main_df)) * 100

In [ ]:
main_df = main_df.dropna()

In [ ]:
main_df.isnull().sum()

In [ ]:
# Q1) When is the best time of day, day of the week, and time of year to fly to minimise delays?

In [ ]:
q1_df = main_df[['Month', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime', 'ArrDelay', 'DepDelay']]

In [ ]:
hourly_df = q1_df[['CRSDepTime', 'CRSArrTime', 'ArrDelay', 'DepDelay']]

In [ ]:
hourly_df['CRSDepTime'].max()

In [ ]:
hourly_df['CRSArrTime'].max()

In [ ]:
hourly_df['CRSDepTime'] = hourly_df['CRSDepTime'] // 100
hourly_df['CRSArrTime'] = hourly_df['CRSArrTime'] // 100

In [ ]:
def hour_format(time):
    if int(time) == 24:
        return int(0)
    else:
        return int(time)

In [ ]:
hourly_df['CRSDepTime'] = hourly_df['CRSDepTime'].apply(hour_format)

In [ ]:
hourly_df['CRSDepTime'].max()

In [ ]:
hourly_df['CRSArrTime'] = hourly_df['CRSArrTime'].apply(hour_format)

In [ ]:
hourly_df['CRSArrTime'].max()

In [ ]:
depdelay_mean = hourly_df.groupby('CRSDepTime')['DepDelay'].mean().reset_index()
arrdelay_mean = hourly_df.groupby('CRSArrTime')['ArrDelay'].mean().reset_index()

In [ ]:
totaldelay = pd.merge(depdelay_mean, arrdelay_mean, left_on = 'CRSDepTime', right_on = 'CRSArrTime').drop('CRSArrTime', axis=1)

In [ ]:
totaldelay = totaldelay.rename(columns = {'CRSDepTime': 'Hours'})

In [ ]:
arrdelay_hourly_mean = totaldelay['ArrDelay']
depdelay_hourly_mean = totaldelay['DepDelay']

In [ ]:
fig, ax = plt.subplots(figsize = (10, 8))
w = 0.125
x = np.arange(len(totaldelay['Hours']))

ax.bar(x - w, arrdelay_hourly_mean, color = 'crimson',
        width = 0.25, edgecolor = 'black', label ='Arrival delay')
ax.bar(x + w, depdelay_hourly_mean, color = 'aliceblue',
        width = 0.25, edgecolor = 'black', label ='Departure delay')

ax.set_xlabel("Hours", fontweight = 'bold', fontsize = 12)
ax.set_ylabel("Hourly mean delay (mins)", fontweight = 'bold', fontsize = 12)
ax.set_xticks(x)
plt.title("Hourly mean arrival and deprature delay over two years", fontweight = 'bold', fontsize = 14)
plt.legend()
plt.show()

In [ ]:
weekly_df = q1_df[['DayOfWeek', 'ArrDelay', 'DepDelay']]

In [ ]:
weekly_df['TotalDelay'] = weekly_df['ArrDelay'] + weekly_df['DepDelay']

In [ ]:
#Weekly delay mean

weekly_mean_delay = weekly_df.groupby('DayOfWeek')['TotalDelay'].mean().reset_index()

In [ ]:
weekly_totaldelay_mean = weekly_mean_delay['TotalDelay']

In [ ]:
#Weekly mean graph

fig, ax = plt.subplots(figsize = (8, 6))
y = np.arange(len(weekly_mean_delay['DayOfWeek']))
labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

color = ['crimson', 'aliceblue']

ax.bar(y, weekly_totaldelay_mean, color = color,
        width = 0.5, edgecolor = 'black')

ax.set_xlabel("Day", fontweight = 'bold', fontsize = 12)
ax.set_ylabel("Weekly mean delay (mins)", fontweight = 'bold', fontsize = 12)
ax.set_xticks(y, labels)
plt.title("Weekly mean of total delay over two years", fontweight = 'bold', fontsize = 14)
plt.show()

In [ ]:
monthly_df = q1_df[['Month', 'ArrDelay', 'DepDelay']]

In [ ]:
monthly_df['TotalDelay'] = monthly_df['ArrDelay'] + monthly_df['DepDelay']

In [ ]:
#Monthly delay mean

monthly_mean_delay = monthly_df.groupby('Month')['TotalDelay'].mean().reset_index()

In [ ]:
monthly_totaldelay_mean = monthly_mean_delay['TotalDelay']

In [ ]:
#Monthly mean graph

fig, ax = plt.subplots(figsize = (12, 8))
z = np.arange(len(monthly_mean_delay['Month']))
labels = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',
          'December']

color = ['crimson', 'aliceblue']

ax.bar(z, monthly_totaldelay_mean, color = color,
        width = 0.5, edgecolor ='black')

ax.set_xlabel("Month", fontweight = 'bold', fontsize = 12)
ax.set_ylabel("Monthly mean delay (mins)", fontweight = 'bold', fontsize = 12)
ax.set_xticks(z, labels)
plt.title("Monthly mean of total delay over two years", fontweight = 'bold', fontsize = 14)
plt.show()

In [ ]:
# Q2) Do older planes suffer more delays?

In [ ]:
plane_data = pd.read_csv('plane-data.csv', encoding = 'latin-1', usecols = ['tailnum', 'year'])

In [ ]:
plane_data.isnull().sum()

In [ ]:
plane_data = plane_data.dropna()

In [ ]:
plane_data = plane_data.sort_values(by = ['year'])

In [ ]:
plane_data = plane_data[(plane_data.year != '') & (plane_data.year != '0000') & (plane_data.year != 'None')]

In [ ]:
plane_data.rename(columns = {'year': 'year of manufacture'}, inplace = True)

In [ ]:
plane_data

In [ ]:
q2_df = main_df[['Year', 'TailNum', 'ArrDelay', 'DepDelay']]

In [ ]:
q2_df = pd.merge(q2_df, plane_data, left_on = 'TailNum', right_on = 'tailnum', how = 'inner', indicator = False)

In [ ]:
q2_df = q2_df.drop(['Year', 'TailNum', 'tailnum'], axis = 1)

In [ ]:
q2_df['Total Delay'] = q2_df['ArrDelay'] + q2_df['DepDelay']

In [ ]:
yearly_plane_mean = q2_df.groupby('year of manufacture')['Total Delay'].mean().reset_index()

In [ ]:
yearly_plane_mean['year of manufacture'] = yearly_plane_mean['year of manufacture'].astype(int)

In [ ]:
q2_56_79 = yearly_plane_mean[yearly_plane_mean['year of manufacture'].between(1956, 1979)]
q2_80_07 = yearly_plane_mean[yearly_plane_mean['year of manufacture'].between(1980, 2007)]

In [ ]:
sns.set_style("darkgrid")

In [ ]:
plt.subplots(figsize = (8, 6))

sns.regplot(data = q2_56_79, x = "year of manufacture", y = "Total Delay", ci = 95, label = '1956-1979')
sns.regplot(data = q2_80_07, x = "year of manufacture", y = "Total Delay", ci = 95, label = '1980-2005')

plt.title("Total Delay mean by year of manufacture", fontdict = {'fontsize': 16, 'fontweight': 'bold'})
plt.xlabel("Year", fontdict = {'fontsize': 14, 'fontweight': 'bold'})
plt.ylabel("Mean Total Delay", fontdict = {'fontsize': 14, 'fontweight': 'bold'})

plt.legend()
plt.show()

In [ ]:
# Q3) How does the number of people (number of flights) flying between different locations change over time?

In [ ]:
q3_df = main_df[['Year', 'Origin', 'Dest']]

In [ ]:
q3_df['Connection'] = q3_df['Origin'] +"-"+ q3_df['Dest']

In [ ]:
q3_df = q3_df.drop(['Origin', 'Dest'], axis = 1)

In [ ]:
q3_df = q3_df.groupby(['Year', 'Connection']).size().reset_index(name = 'Counts')

In [ ]:
q3_df['Connection'] = q3_df['Connection'].str.split('-').apply(lambda x: '-'.join(sorted(x)))

In [ ]:
q3_df = q3_df.groupby(['Year', 'Connection']).sum().reset_index()

In [ ]:
q3_df = q3_df.pivot(index = 'Connection', columns = 'Year', values = 'Counts')

In [ ]:
q3_df = q3_df.dropna()

In [ ]:
q3_df['Total'] = q3_df.sum(axis = 1)

In [ ]:
filtered_q3_df = q3_df.loc[(q3_df['Total'] > 100) & (q3_df['Total'] < 500)]

In [ ]:
filtered_q3_df = filtered_q3_df.sort_values(by = ['Total'], ascending = True)

In [ ]:
select_airports = filtered_q3_df.drop(['Total'], axis = 1)

In [ ]:
select5_airports = select_airports.head(5)

In [ ]:
select5_airports = select5_airports.div(select5_airports.sum(axis = 1), axis = 0).mul(100).round(1)

In [ ]:
ax = select5_airports.plot(kind = 'bar', stacked = True, colormap = 'tab10', figsize = (8,6))

ax.set_xlabel("Connection", fontdict = {'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel("Percentage", fontdict = {'fontsize': 14, 'fontweight': 'bold'})
ax.set_title("Percentage stacked bar plot of 5 connections over two years", 
             fontdict = {'fontsize': 14, 'fontweight': 'bold'})
plt.xticks(rotation = 0)
ax.legend(loc = 'center left', bbox_to_anchor = (1.0, 0.5))
plt.show()

In [ ]:
# Q4) Can you detect cascading failures as delays in one airport create delays in others?

In [ ]:
q4_df = main_df[['Origin', 'Dest', 'ArrDelay', 'DepDelay']]

In [ ]:
q4_df.isnull().sum()

In [ ]:
q4_df = q4_df.dropna()

In [ ]:
q4_df['Total delay'] = q4_df['ArrDelay'] + q4_df['DepDelay']

In [ ]:
top5_airports = q4_df.groupby(['Origin'])['Total delay'].sum().nlargest(5).index

In [ ]:
top5_airport_delays = q4_df[q4_df['Origin'].isin(top5_airports) & q4_df['Dest'].isin(top5_airports)]

In [ ]:
delay_matrix = pd.pivot_table(top5_airport_delays, values = 'Total delay', index = ['Origin'], columns = ['Dest'], 
                              aggfunc = np.mean)

In [ ]:
corr_matrix = delay_matrix.corr()

In [ ]:
mask = np.zeros_like(corr_matrix, dtype = bool)
mask[np.triu_indices_from(mask)] = True

In [ ]:
sns.set(font_scale = 1.3)
plt.figure(figsize = (10,6))
sns.heatmap(corr_matrix, mask = mask, annot = True, cmap = 'RdBu_r', vmin = -1, vmax = 1, square = True,
            xticklabels = corr_matrix.columns, yticklabels = corr_matrix.index, cbar_kws={"shrink": 0.5})

plt.yticks(rotation = 0)
plt.title('Triangle correlation heatmap for top 5 airports by total delay', fontsize = 14, fontweight = 'bold')
plt.xlabel('Destination Airport', fontsize = 14, fontweight = 'bold')
plt.ylabel('Origin Airport', fontsize = 14, fontweight = 'bold')

plt.show()

In [ ]:
# Q5) Use the available variables to construct a model that predicts delays

In [ ]:
%matplotlib inline
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
q5_df = main_df[['CRSDepTime', 'CRSArrTime', 'DepTime', 'ArrTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest']]

In [ ]:
q5_df

In [ ]:
q5_df.isnull().sum()

In [ ]:
sample_df = q5_df.sample(200000)

In [ ]:
sample_df['Total Delay'] = sample_df['ArrDelay'] + sample_df['DepDelay']

In [ ]:
sample_df['TotalDelay'] = np.where(sample_df['Total Delay'] > 0, 1, 0)

In [ ]:
sample_df = sample_df.drop(['ArrDelay', 'DepDelay', 'Total Delay'], axis = 1)

In [ ]:
features = ['CRSDepTime', 'CRSArrTime', 'DepTime', 'ArrTime', 'Origin', 'Dest']

X = sample_df[features].copy()
Y = sample_df['TotalDelay']

In [ ]:
numerical_features = ['CRSDepTime', 'CRSArrTime', 'DepTime', 'ArrTime']

In [ ]:
numerical_transformer = Pipeline(steps=[
 ('imputer', SimpleImputer()),
 ('scaler', StandardScaler())])

In [ ]:
categorical_features = ['Origin', 'Dest']

In [ ]:
categorical_transformer = Pipeline(steps=[
 ('imputer', SimpleImputer()),
 ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
data_transformer = ColumnTransformer(
 transformers=[
 ('num', numerical_transformer, numerical_features),
 ('cat', categorical_transformer, categorical_features)])

In [ ]:
pipe_lr = Pipeline(steps=[('data_transformer', data_transformer),
 ('pipe_lr', LogisticRegression(max_iter = 10000))])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,random_state = 1)

In [ ]:
X_train.values.reshape(-1, 1)

In [ ]:
y_train.values.reshape(-1, 1)

In [ ]:
param_grid = {
 'data_transformer__num__imputer__strategy': ['mean', 'median'],
 'data_transformer__cat__imputer__strategy': ['constant','most_frequent']
}

In [ ]:
grid_lr = GridSearchCV(pipe_lr, param_grid=param_grid, n_jobs=-1)

In [ ]:
%%timeit

grid_lr.fit(X_train, y_train)

In [ ]:
pipe_gdb = Pipeline(steps=[('data_transformer', data_transformer),
 ('pipe_gdb',GradientBoostingClassifier(random_state=2))])

In [ ]:
grid_gdb = GridSearchCV(pipe_gdb, param_grid=param_grid, n_jobs=-1)

In [ ]:
%%timeit

grid_gdb.fit(X_train, y_train)

In [ ]:
ax = plt.gca()
plot_roc_curve(grid_lr, X_test, y_test, ax = ax, name='Logistic Regression')
plot_roc_curve(grid_gdb, X_test, y_test, ax = ax, name='Gradient Boosting')
plt.show()